# 数据读取与处理

In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

dataPath = 'C:/Users/zhang/mine/finTech/2021/data/A'

trainDf = pd.read_csv(dataPath + '/train_v1.csv')
wkdDf = pd.read_csv(dataPath + '/wkd_v1.csv')
testDf = pd.read_csv(dataPath + '/test_v1_periods.csv')
testDayDf = pd.read_csv(dataPath + '/test_v1_day.csv')

wkdDf = wkdDf.rename(columns={'ORIG_DT':'date'})

######### 这里数据官方已清洗过
trainDf = trainDf.merge(wkdDf, on = ['date'], how = 'left')
testDf = testDf.merge(wkdDf, on = ['date'], how = 'left')
testDayDf = testDayDf.merge(wkdDf, on = ['date'], how = 'left')

trainDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 695520 entries, 0 to 695519
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        695520 non-null  object
 1   post_id     695520 non-null  object
 2   biz_type    695520 non-null  object
 3   periods     695520 non-null  int64 
 4   amount      695520 non-null  int64 
 5   WKD_TYP_CD  695520 non-null  object
dtypes: int64(2), object(4)
memory usage: 37.1+ MB


In [2]:
## 提取岗位以日为粒度的业务量
def get_jobDayAmount(df):
    outputDf = df.groupby(['date', 'post_id', 'WKD_TYP_CD'], as_index = False)['amount'].sum()
    # outputDf = outputDf.sort_values(by = ['date', 'post_id'], axis = 0, ascending = True)
    return outputDf

trainDayDf = get_jobDayAmount(trainDf)
trainDayDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2070 entries, 0 to 2069
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        2070 non-null   object
 1   post_id     2070 non-null   object
 2   WKD_TYP_CD  2070 non-null   object
 3   amount      2070 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 80.9+ KB


In [3]:
## 提取岗位以0.5小时为粒度的业务量
def get_jobPeriodAmount(df):
    outputDf = df.groupby(['date', 'post_id', 'periods', 'WKD_TYP_CD'], as_index = False)['amount'].sum()
    # outputDf = outputDf.sort_values(by = ['date', 'post_id', 'periods'], axis = 0, ascending = True)
    return outputDf

trainDf = get_jobPeriodAmount(trainDf)
trainDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99360 entries, 0 to 99359
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        99360 non-null  object
 1   post_id     99360 non-null  object
 2   periods     99360 non-null  int64 
 3   WKD_TYP_CD  99360 non-null  object
 4   amount      99360 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 4.5+ MB


In [5]:
## 提取时间特征，此处以年、月、日作为变量
def getDateDf(df):
    df['date'] = pd.to_datetime(df['date'], format = '%Y/%m/%d')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df.drop(['date'], axis=1, inplace=True)
    return df

trainDf = getDateDf(trainDf)
trainDayDf = getDateDf(trainDayDf)
testDf = getDateDf(testDf)
testDayDf = getDateDf(testDayDf)
trainDf.head()

,post_id,periods,WKD_TYP_CD,amount,bizCount,year,month,day
0,A,1,NH,0,13,2018,1,1
1,A,2,NH,0,13,2018,1,1
2,A,3,NH,0,13,2018,1,1
3,A,4,NH,0,13,2018,1,1
4,A,5,NH,0,13,2018,1,1


# 模型部分

In [6]:
## 载入所需库
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold,GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,accuracy_score, \
                            precision_score,recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [7]:
## 将字符向量转化为值向量
def labelEncoder_df(df, features):
    for i in features:
        encoder = LabelEncoder()
        df[i] = encoder.fit_transform(df[i])
        
## 节省内存
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## 以0.5h为粒度

In [8]:
trainCols = trainDf.columns.tolist()
testCols = testDf.columns.tolist()

In [9]:
trainDf['isTest'] = -1
testDf['isTest'] = 1
totalDf = pd.concat([trainDf, testDf])

In [10]:
cols = ['post_id', 'WKD_TYP_CD']
for col in cols:
    if totalDf[col].dtype == 'object':
        totalDf[col] = totalDf[col].astype(str)
labelEncoder_df(totalDf, cols)

In [11]:
trainDf = totalDf[totalDf['isTest'] == -1]
trainDf = trainDf[trainCols]
testDf = totalDf[totalDf['isTest'] == 1]
testDf = testDf[testCols]

trainDf['amount'] = trainDf['amount'].astype(int)

trainDf = reduce_mem_usage(trainDf)
testDf = reduce_mem_usage(testDf)

Mem. usage decreased to  1.80 Mb (68.3% reduction)
Mem. usage decreased to  0.06 Mb (58.9% reduction)


In [12]:
trainDf.info()
trainDf.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99360 entries, 0 to 99359
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   post_id     99360 non-null  int8   
 1   periods     99360 non-null  int8   
 2   WKD_TYP_CD  99360 non-null  int8   
 3   amount      99360 non-null  int16  
 4   bizCount    99360 non-null  float16
 5   year        99360 non-null  int16  
 6   month       99360 non-null  int8   
 7   day         99360 non-null  int8   
dtypes: float16(1), int16(2), int8(5)
memory usage: 1.8 MB


,post_id,periods,WKD_TYP_CD,amount,bizCount,year,month,day
0,0,1,0,0,13.0,2018,1,1
1,0,2,0,0,13.0,2018,1,1
2,0,3,0,0,13.0,2018,1,1
3,0,4,0,0,13.0,2018,1,1
4,0,5,0,0,13.0,2018,1,1


## 模型

In [19]:
dfX = trainDf.drop(['amount'], axis = 1)
dfY = trainDf['amount']

trainX, valX, trainY, valY = train_test_split(dfX, dfY, test_size=0.2, random_state = 42)

In [20]:
################
## 评价函数 
def calMAPE(y, y_predict, name=None):
    y = y.to_numpy()
    y_predict = y_predict.to_numpy()
    temp = np.abs((y - y_predict) / y)
    temp = temp[~np.isinf(temp)]
    MAPE = np.nanmean(temp, axis=0)
    # MAPE = np.mean(np.abs((y - y_predict) / y)) * 100
    if name:
        print(name,':')
    print(f'MAPE is: {MAPE}')
    print('- - - - - - ')
    return MAPE

In [42]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [43]:
from sklearn.model_selection import GridSearchCV

rf_cfl = RandomForestRegressor(n_estimators=120).fit(trainX, trainY)

param_grid = {
            'n_estimators': [50, 100, 200, 250],
            # 'max_features': [2, 3],
            'min_samples_leaf': [1, 2, 4],
            'min_samples_split': [2, 5, 10],
            }
CV_rnd_cfl = GridSearchCV(estimator = rf_cfl, param_grid = param_grid, scoring = 'neg_mean_squared_error', verbose = 10, n_jobs = -1)
CV_rnd_cfl.fit(trainX, trainY)

best_parameters = CV_rnd_cfl.best_params_
print("The best parameters: ", best_parameters)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 16.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 21.9min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 26.9min finished


The best parameters:  {'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250}


## 以0.5h为粒度的task2

In [44]:
random_seed = 2021
np.random.seed(2)
    
rf_cfl = RandomForestRegressor(n_estimators = 250,
                               min_samples_split = 2,
                               min_samples_leaf = 1,
                               #max_features = 3
                              )
rf_cfl.fit(trainX, trainY)
y_rf = rf_cfl.predict(valX)
y_rf = y_rf.astype(int)
y_rf = pd.Series(y_rf.tolist())
y_rf.name = 'amount'
print("Train Score:%f" % rf_cfl.score(trainX, trainY))
print("Val Score:%f" % rf_cfl.score(valX, valY))

Train Score:0.998275
Val Score:0.984758


In [45]:
MAPE2 = calMAPE(valY, y_rf)
print(MAPE2)

MAPE is: 0.2947687624700864
- - - - - - 
0.2947687624700864


## 以天为粒度

In [46]:
trainDayCols = trainDayDf.columns.tolist()
testDayCols = testDayDf.columns.tolist()

In [47]:
trainDayDf['isTest'] = -1
testDayDf['isTest'] = 1
totalDayDf = pd.concat([trainDayDf, testDayDf])

In [48]:
cols = ['post_id', 'WKD_TYP_CD']
for col in cols:
    if totalDayDf[col].dtype == 'object':
        totalDayDf[col] = totalDayDf[col].astype(str)
labelEncoder_df(totalDayDf, cols)

In [49]:
trainDayDf = totalDayDf[totalDayDf['isTest'] == -1]
trainDayDf = trainDayDf[trainDayCols]
testDayDf = totalDayDf[totalDayDf['isTest'] == 1]
testDayDf = testDayDf[testDayCols]

trainDayDf['amount'] = trainDayDf['amount'].astype(int)

trainDayDf = reduce_mem_usage(trainDayDf)
testDayDf = reduce_mem_usage(testDayDf)

Mem. usage decreased to  0.04 Mb (41.2% reduction)
Mem. usage decreased to  0.00 Mb (38.9% reduction)


In [50]:
dfDayX = trainDayDf.drop(['amount'], axis = 1)
dfDayY = trainDayDf['amount']

trainDayX, valDayX, trainDayY, valDayY = train_test_split(dfDayX, dfDayY, test_size=0.2, random_state = 42)

In [51]:
random_seed = 2021
np.random.seed(2)
    
rfDay_cfl = RandomForestRegressor(n_estimators = 250,
                               min_samples_split = 2,
                               min_samples_leaf = 1,
                                 )
rfDay_cfl.fit(trainDayX, trainDayY)
y_rfDay = rfDay_cfl.predict(valDayX)
y_rfDay = y_rfDay.astype(int)
y_rfDay = pd.Series(y_rfDay.tolist())
y_rfDay.name = 'amount'
print("Train Score:%f" % rfDay_cfl.score(trainDayX, trainDayY))
print("Val Score:%f" % rfDay_cfl.score(valDayX, valDayY))

Train Score:0.990938
Val Score:0.934749


In [52]:
MAPE1 = calMAPE(valDayY, y_rfDay)
print(MAPE1)

MAPE is: 0.26511586233516204
- - - - - - 
0.26511586233516204


In [53]:
MAPE = 0.7 * MAPE1 + 0.3 * MAPE2
print('total MAPE is:', MAPE)

total MAPE is: 0.2740117323756393


## 直接基于以0.5h为粒度的模型预测task1

In [17]:
valDayDf = pd.concat([valX, valY], axis = 1)
valDayDf = valDayDf.reset_index(drop = True)
valDayDf.head()

,post_id,periods,WKD_TYP_CD,bizCount,year,month,day,amount
0,0,37,3,13.0,2018,1,4,1
1,0,23,3,13.0,2019,12,25,0
2,1,34,3,1.0,2019,7,29,372
3,0,44,3,13.0,2019,8,5,0
4,0,46,1,13.0,2020,7,12,0


In [60]:
rfDayDf = pd.concat([valX.reset_index(drop = True), y_rf], axis = 1)
rfDayDf = rfDayDf.reset_index(drop = True)
rfDayDf.head()

,post_id,periods,WKD_TYP_CD,bizCount,year,month,day,amount
0,0,37,3,13.0,2018,1,4,0
1,0,23,3,13.0,2019,12,25,1494
2,1,34,3,1.0,2019,7,29,336
3,0,44,3,13.0,2019,8,5,0
4,0,46,1,13.0,2020,7,12,0


In [19]:
from datetime import datetime
def repairDate(df):
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
    df['date'] = df['date'].apply(lambda x: datetime.strftime(x, format = '%Y/%#m/%#d'))
    df.drop(['year', 'month', 'day'], axis = 1, inplace = True)
    return df

In [20]:
valDayDf = repairDate(valDayDf)
valDayDf.head()

,post_id,periods,WKD_TYP_CD,bizCount,amount,date
0,0,37,3,13.0,1,2018/1/4
1,0,23,3,13.0,0,2019/12/25
2,1,34,3,1.0,372,2019/7/29
3,0,44,3,13.0,0,2019/8/5
4,0,46,1,13.0,0,2020/7/12


In [61]:
rfDayDf = repairDate(rfDayDf)
rfDayDf.head()

,post_id,periods,WKD_TYP_CD,bizCount,amount,date
0,0,37,3,13.0,0,2018/1/4
1,0,23,3,13.0,1494,2019/12/25
2,1,34,3,1.0,336,2019/7/29
3,0,44,3,13.0,0,2019/8/5
4,0,46,1,13.0,0,2020/7/12


In [62]:
valDayDf = get_jobDayAmount(valDayDf)
rfDayDf = get_jobDayAmount(rfDayDf)

In [23]:
valDayDf.head()

,date,post_id,WKD_TYP_CD,amount
0,2018/1/1,0,0,590
1,2018/1/1,1,0,0
2,2018/1/10,0,3,9454
3,2018/1/10,1,3,785
4,2018/1/11,0,3,6419


In [24]:
rfDayDf.head()

,date,post_id,WKD_TYP_CD,amount
0,2018/1/1,0,0,651
1,2018/1/1,1,0,0
2,2018/1/10,0,3,9441
3,2018/1/10,1,3,800
4,2018/1/11,0,3,6823


In [25]:
MAPE1 = calMAPE(valDayDf['amount'], rfDayDf['amount'])
print(MAPE1)

MAPE is: 0.10281209224720003
- - - - - - 
0.10281209224720003


In [26]:
MAPE = 0.7 * MAPE1 + 0.3 * MAPE2
print('total MAPE is:', MAPE)

total MAPE is: 0.16174836436109552


## v1
2021/05/01 v1 0.18498 第14

In [55]:
from xgboost.sklearn import XGBRegressor

In [56]:
random_seed = 2021
np.random.seed(2)
    
xgb_cfl = XGBRegressor()
xgb_cfl.fit(trainX, trainY)
y_xgb = xgb_cfl.predict(valX)
y_xgb = y_xgb.astype(int)
y_xgb = pd.Series(y_xgb.tolist())
y_xgb.name = 'amount'
print("Train Score:%f" % xgb_cfl.score(trainX, trainY))
print("Val Score:%f" % xgb_cfl.score(valX, valY))

Train Score:0.979575
Val Score:0.970916


In [63]:
MAPE2 = calMAPE(valY, y_xgb)
print(MAPE2)

MAPE is: 0.9228565016534319
- - - - - - 
0.9228565016534319


## task 1

In [64]:
xgbDayDf = pd.concat([valX.reset_index(drop = True), y_xgb], axis = 1)
xgbDayDf = xgbDayDf.reset_index(drop = True)
xgbDayDf.head()

,post_id,periods,WKD_TYP_CD,bizCount,year,month,day,amount
0,0,37,3,13.0,2018,1,4,-52
1,0,23,3,13.0,2019,12,25,870
2,1,34,3,1.0,2019,7,29,323
3,0,44,3,13.0,2019,8,5,0
4,0,46,1,13.0,2020,7,12,2


In [65]:
xgbDayDf = repairDate(xgbDayDf)
xgbDayDf.head()

,post_id,periods,WKD_TYP_CD,bizCount,amount,date
0,0,37,3,13.0,-52,2018/1/4
1,0,23,3,13.0,870,2019/12/25
2,1,34,3,1.0,323,2019/7/29
3,0,44,3,13.0,0,2019/8/5
4,0,46,1,13.0,2,2020/7/12


In [67]:
xgbDayDf = get_jobDayAmount(xgbDayDf)

In [68]:
MAPE1 = calMAPE(valDayDf['amount'], xgbDayDf['amount'])
print(MAPE1)


MAPE is: 0.6016117001247102
- - - - - - 
0.6016117001247102
